In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.13.0


Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

Create model in class version

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() #RandomNormal에서 glorot_uniform으로 바꿔준다

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

Create model in function version

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

Define data & Hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

Restore checkpoint & start train or test phrase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer("C:/Users/johnk/logs/")
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        #checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\johnk\anaconda3\Lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 0.9913, train_loss: 2.10431099, train_accuracy: 0.3047, test_Accuracy: 0.1993
Epoch: [ 0] [    1/  468] time: 1.3973, train_loss: 2.04646063, train_accuracy: 0.5078, test_Accuracy: 0.4262
Epoch: [ 0] [    2/  468] time: 1.8161, train_loss: 1.96768320, train_accuracy: 0.5625, test_Accuracy: 0.5368
Epoch: [ 0] [    3/  468] time: 2.1812, train_loss: 1.86466444, train_accuracy: 0.6016, test_Accuracy: 0.5695
Epoch: [ 0] [    4/  468] time: 2.4405, train_loss: 1.80678344, train_accuracy: 0.6094, test_Accuracy: 0.5817
Epoch: [ 0] [    5/  468] time: 2.6379, train_loss: 1.67909861, train_accuracy: 0.6094, test_Accuracy: 0.6239
Epoch: [ 0] [    6/  468] time: 2.8484, train_loss: 1.61333680, train_accuracy: 0.6406, test_Accuracy: 0.6606
Epoch: [ 0] [    7/  468] time: 3.0897, train_loss: 1.47643089, train_accuracy: 0.6641, test_Accuracy: 0.6991
Epoch: [ 0] [    8/  468] time: 3.3331, train_loss: 1.44979024, train_accuracy: 0.6719, test_Accuracy: 0.7331
Epoch: [ 0

Epoch: [ 0] [   76/  468] time: 18.8635, train_loss: 0.30038244, train_accuracy: 0.9062, test_Accuracy: 0.9201
Epoch: [ 0] [   77/  468] time: 19.0500, train_loss: 0.22252719, train_accuracy: 0.9297, test_Accuracy: 0.9194
Epoch: [ 0] [   78/  468] time: 19.2435, train_loss: 0.26979256, train_accuracy: 0.9219, test_Accuracy: 0.9185
Epoch: [ 0] [   79/  468] time: 19.4599, train_loss: 0.25146493, train_accuracy: 0.9297, test_Accuracy: 0.9176
Epoch: [ 0] [   80/  468] time: 19.6853, train_loss: 0.39499700, train_accuracy: 0.8750, test_Accuracy: 0.9209
Epoch: [ 0] [   81/  468] time: 19.8997, train_loss: 0.31111628, train_accuracy: 0.8750, test_Accuracy: 0.9201
Epoch: [ 0] [   82/  468] time: 20.0932, train_loss: 0.22172850, train_accuracy: 0.9609, test_Accuracy: 0.9185
Epoch: [ 0] [   83/  468] time: 20.2797, train_loss: 0.34056666, train_accuracy: 0.9375, test_Accuracy: 0.9164
Epoch: [ 0] [   84/  468] time: 20.4912, train_loss: 0.21858792, train_accuracy: 0.9141, test_Accuracy: 0.9166
E

Epoch: [ 0] [  150/  468] time: 33.4405, train_loss: 0.18388517, train_accuracy: 0.9453, test_Accuracy: 0.9380
Epoch: [ 0] [  151/  468] time: 33.6220, train_loss: 0.31512821, train_accuracy: 0.9062, test_Accuracy: 0.9381
Epoch: [ 0] [  152/  468] time: 33.8185, train_loss: 0.20650214, train_accuracy: 0.9297, test_Accuracy: 0.9393
Epoch: [ 0] [  153/  468] time: 34.0319, train_loss: 0.17647219, train_accuracy: 0.9609, test_Accuracy: 0.9394
Epoch: [ 0] [  154/  468] time: 34.2553, train_loss: 0.14210889, train_accuracy: 0.9609, test_Accuracy: 0.9366
Epoch: [ 0] [  155/  468] time: 34.4398, train_loss: 0.18404110, train_accuracy: 0.9531, test_Accuracy: 0.9333
Epoch: [ 0] [  156/  468] time: 34.6263, train_loss: 0.14226954, train_accuracy: 0.9609, test_Accuracy: 0.9289
Epoch: [ 0] [  157/  468] time: 34.8218, train_loss: 0.29880363, train_accuracy: 0.9453, test_Accuracy: 0.9271
Epoch: [ 0] [  158/  468] time: 34.9973, train_loss: 0.16914913, train_accuracy: 0.9609, test_Accuracy: 0.9296
E

Epoch: [ 0] [  224/  468] time: 47.7642, train_loss: 0.23285316, train_accuracy: 0.9375, test_Accuracy: 0.9497
Epoch: [ 0] [  225/  468] time: 47.9397, train_loss: 0.15400067, train_accuracy: 0.9609, test_Accuracy: 0.9492
Epoch: [ 0] [  226/  468] time: 48.1352, train_loss: 0.18653494, train_accuracy: 0.9297, test_Accuracy: 0.9469
Epoch: [ 0] [  227/  468] time: 48.3247, train_loss: 0.15703905, train_accuracy: 0.9531, test_Accuracy: 0.9469
Epoch: [ 0] [  228/  468] time: 48.5191, train_loss: 0.13785726, train_accuracy: 0.9609, test_Accuracy: 0.9473
Epoch: [ 0] [  229/  468] time: 48.7136, train_loss: 0.18660435, train_accuracy: 0.9453, test_Accuracy: 0.9473
Epoch: [ 0] [  230/  468] time: 48.9101, train_loss: 0.20364779, train_accuracy: 0.9453, test_Accuracy: 0.9461
Epoch: [ 0] [  231/  468] time: 49.1096, train_loss: 0.14046139, train_accuracy: 0.9688, test_Accuracy: 0.9433
Epoch: [ 0] [  232/  468] time: 49.3080, train_loss: 0.25137240, train_accuracy: 0.9375, test_Accuracy: 0.9422
E

Epoch: [ 0] [  298/  468] time: 62.0410, train_loss: 0.09613699, train_accuracy: 0.9688, test_Accuracy: 0.9554
Epoch: [ 0] [  299/  468] time: 62.2773, train_loss: 0.09278439, train_accuracy: 0.9766, test_Accuracy: 0.9555
Epoch: [ 0] [  300/  468] time: 62.4898, train_loss: 0.20110156, train_accuracy: 0.9375, test_Accuracy: 0.9558
Epoch: [ 0] [  301/  468] time: 62.6723, train_loss: 0.08415523, train_accuracy: 0.9609, test_Accuracy: 0.9562
Epoch: [ 0] [  302/  468] time: 62.8548, train_loss: 0.18967715, train_accuracy: 0.9531, test_Accuracy: 0.9545
Epoch: [ 0] [  303/  468] time: 63.0413, train_loss: 0.14498089, train_accuracy: 0.9375, test_Accuracy: 0.9508
Epoch: [ 0] [  304/  468] time: 63.2547, train_loss: 0.09890333, train_accuracy: 0.9688, test_Accuracy: 0.9474
Epoch: [ 0] [  305/  468] time: 63.4442, train_loss: 0.12353162, train_accuracy: 0.9609, test_Accuracy: 0.9474
Epoch: [ 0] [  306/  468] time: 63.6387, train_loss: 0.19827953, train_accuracy: 0.9375, test_Accuracy: 0.9485
E

Epoch: [ 0] [  372/  468] time: 76.2858, train_loss: 0.14290677, train_accuracy: 0.9531, test_Accuracy: 0.9590
Epoch: [ 0] [  373/  468] time: 76.4873, train_loss: 0.08471024, train_accuracy: 0.9766, test_Accuracy: 0.9587
Epoch: [ 0] [  374/  468] time: 76.7047, train_loss: 0.10465880, train_accuracy: 0.9688, test_Accuracy: 0.9593
Epoch: [ 0] [  375/  468] time: 76.9002, train_loss: 0.20225061, train_accuracy: 0.9531, test_Accuracy: 0.9616
Epoch: [ 0] [  376/  468] time: 77.0917, train_loss: 0.07973017, train_accuracy: 0.9844, test_Accuracy: 0.9618
Epoch: [ 0] [  377/  468] time: 77.2732, train_loss: 0.14381796, train_accuracy: 0.9688, test_Accuracy: 0.9605
Epoch: [ 0] [  378/  468] time: 77.4497, train_loss: 0.08904716, train_accuracy: 0.9844, test_Accuracy: 0.9571
Epoch: [ 0] [  379/  468] time: 77.6512, train_loss: 0.08301747, train_accuracy: 0.9844, test_Accuracy: 0.9560
Epoch: [ 0] [  380/  468] time: 77.8536, train_loss: 0.11206523, train_accuracy: 0.9688, test_Accuracy: 0.9540
E

Epoch: [ 0] [  446/  468] time: 90.4360, train_loss: 0.20374516, train_accuracy: 0.9453, test_Accuracy: 0.9633
Epoch: [ 0] [  447/  468] time: 90.6235, train_loss: 0.16405286, train_accuracy: 0.9531, test_Accuracy: 0.9640
Epoch: [ 0] [  448/  468] time: 90.8090, train_loss: 0.09242697, train_accuracy: 0.9844, test_Accuracy: 0.9622
Epoch: [ 0] [  449/  468] time: 90.9995, train_loss: 0.12817463, train_accuracy: 0.9453, test_Accuracy: 0.9625
Epoch: [ 0] [  450/  468] time: 91.2009, train_loss: 0.13923340, train_accuracy: 0.9609, test_Accuracy: 0.9639
Epoch: [ 0] [  451/  468] time: 91.3934, train_loss: 0.12175159, train_accuracy: 0.9688, test_Accuracy: 0.9649
Epoch: [ 0] [  452/  468] time: 91.5869, train_loss: 0.16308181, train_accuracy: 0.9609, test_Accuracy: 0.9650
Epoch: [ 0] [  453/  468] time: 91.7764, train_loss: 0.11760125, train_accuracy: 0.9688, test_Accuracy: 0.9644
Epoch: [ 0] [  454/  468] time: 91.9778, train_loss: 0.06070518, train_accuracy: 0.9766, test_Accuracy: 0.9650
E

In [ ]:
Test accuracy = 95.8% 